In [1]:
# 11:25 AM
# 11:40 AM
# 11:45 AM
import os; print(os.path.dirname(os.getcwd()).split('\\')[-1])

01-Ins_Review_Eval_Metrics


# Undersampling

Implement the cluster centroids and random undersampling techniques with the credit card default data. Then estimate a logistic regression model and report the classification evaluation metrics from both sampling methods. 

ln_balance_limit is the log of the maximum balance they can have on the card; 1 is female, 0 male for sex; the education is denoted: 1 = graduate school; 2 = university; 3 = high school; 4 = others; 1 is married and 0 single for marriage; default_next_month is whether the person defaults in the following month (1 yes, 0 no).

In [2]:
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
file_path="C:/Users/CS_Knit_tinK_SC/Documents/My Data Sources/112021/cc_default.csv"
data = Path(file_path)
df = pd.read_csv(data)

<font color='red'>Comment</font>

This next line...it is called something. What is it? What does it do?

In [4]:
x_cols = [i for i in df.columns if i not in ('ID', 'default_next_month')]
X = df[x_cols]
y = df['default_next_month']

In [5]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Random Undersampling

In [6]:
# Undersample the data using `RandomUnderSampler`
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 4968, 1: 4968})

<font color='red'>Comment</font>

Perfectly equalized.

In [7]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [8]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[3732, 2100],
       [ 740,  928]], dtype=int64)

<font color='red'>Comment</font>

Look at those FPs!

In [9]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.5981363057701987

<font color='red'>Comment</font>

Random oversampling accuracy = 0.908

This might signal that the additional data used in the oversampling was needed to train the model. 

Is it possible to collect more data?

In [10]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.64      0.56      0.72      0.60      0.36      5832
          1       0.31      0.56      0.64      0.40      0.60      0.35      1668

avg / total       0.72      0.62      0.57      0.65      0.60      0.36      7500



## Cluster Centroid Undersampling

<font color='red'>Comment</font>

> Method that under samples the majority class by replacing a cluster of majority samples by the cluster centroid of a KMeans algorithm. This algorithm keeps N majority samples by fitting the KMeans algorithm with N cluster to the majority class and using the coordinates of the N cluster centroids as the new majority samples.

The idea is that the sampled data should be more representative of the majority class...at least more representative than random sampling.

https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.ClusterCentroids.html

In [11]:
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({0: 4968, 1: 4968})

In [12]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[2827, 3005],
       [ 607, 1061]], dtype=int64)

In [14]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.5604152480484748

<font color='red'>Comment</font>

Random undersampling accuracy = 0.598.

Not sure why...

Q) What sampling method would you use for THIS dataset?

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.48      0.64      0.61      0.56      0.30      5832
          1       0.26      0.64      0.48      0.37      0.56      0.31      1668

avg / total       0.70      0.52      0.60      0.56      0.56      0.31      7500



<font color='red'>Comment</font>

**Key question - if you don't have much data, should you oversample or undersample?**

# Break

you should probably oversample. Undersampling would remove what few precious data points you have.

Next, we look at combining over and undersampling.